# Regression Week 5: Feature Selection and LASSO (Interpretation)

In this notebook, you will use LASSO to select features, building on a pre-implemented solver for LASSO (using GraphLab Create, though you can use other solvers). You will:
* Run LASSO with different L1 penalties.
* Choose best L1 penalty using a validation set.
* Choose best L1 penalty using a validation set, with additional constraint on the size of subset.

In the second notebook, you will implement your own LASSO solver, using coordinate descent. 

# Fire up Graphlab Create

In [1]:
import graphlab
import numpy as np

# Load in house sales data

Dataset is from house sales in King County, the region where the city of Seattle, WA is located.

In [2]:
sales = graphlab.SFrame('kc_house_data.gl/')

[INFO] graphlab.cython.cy_server: GraphLab Create v2.1 started. Logging: /tmp/graphlab_server_1555968420.log


This non-commercial license of GraphLab Create for academic use is assigned to arian@freightpath.io and will expire on April 04, 2020.


# Create new features

As in Week 2, we consider features that are some transformations of inputs.

In [3]:
from math import log, sqrt
sales['sqft_living_sqrt'] = sales['sqft_living'].apply(sqrt)
sales['sqft_lot_sqrt'] = sales['sqft_lot'].apply(sqrt)
sales['bedrooms_square'] = sales['bedrooms']*sales['bedrooms']

# In the dataset, 'floors' was defined with type string, 
# so we'll convert them to float, before creating a new feature.
sales['floors'] = sales['floors'].astype(float) 
sales['floors_square'] = sales['floors']*sales['floors']

* Squaring bedrooms will increase the separation between not many bedrooms (e.g. 1) and lots of bedrooms (e.g. 4) since 1^2 = 1 but 4^2 = 16. Consequently this variable will mostly affect houses with many bedrooms.
* On the other hand, taking square root of sqft_living will decrease the separation between big house and small house. The owner may not be exactly twice as happy for getting a house that is twice as big.

# Learn regression weights with L1 penalty

Let us fit a model with all the features available, plus the features we just created above.

In [4]:
all_features = ['bedrooms', 'bedrooms_square',
            'bathrooms',
            'sqft_living', 'sqft_living_sqrt',
            'sqft_lot', 'sqft_lot_sqrt',
            'floors', 'floors_square',
            'waterfront', 'view', 'condition', 'grade',
            'sqft_above',
            'sqft_basement',
            'yr_built', 'yr_renovated']

Applying L1 penalty requires adding an extra parameter (`l1_penalty`) to the linear regression call in GraphLab Create. (Other tools may have separate implementations of LASSO.)  Note that it's important to set `l2_penalty=0` to ensure we don't introduce an additional L2 penalty.

In [5]:
model_all = graphlab.linear_regression.create(sales, target='price', features=all_features,
                                              validation_set=None, 
                                              l2_penalty=0., l1_penalty=1e10)

Linear regression:

--------------------------------------------------------

Number of examples          : 21613

Number of features          : 17

Number of unpacked features : 17

...

| 4         | 5        | 0.000002  | 1.295312     | 6847166.848958     | 384842.383767 |

| 5         | 6        | 0.000002  | 1.314741     | 6869667.895833     | 385998.458623 |

| 6         | 7        | 0.000002  | 1.333772     | 6847177.773672     | 380824.455891 |

+-----------+----------+-----------+--------------+--------------------+---------------+

TERMINATED: Iteration limit reached.

Find what features had non-zero weight.

In [6]:
non_zero_features = model_all.coefficients[model_all.coefficients['value'] != 0]
print '%d features out of %d were non-zero after L1 regularization.' %(len(non_zero_features), len(all_features))
non_zero_features

This model may not be optimal. To improve it, consider increasing `max_iterations`.

6 features out of 17 were non-zero after L1 regularization.


name,index,value,stderr
(intercept),None,274873.05595,None
bathrooms,None,8468.53108691,None
sqft_living,None,24.4207209824,None
sqft_living_sqrt,None,350.060553386,None
grade,None,842.068034898,None
sqft_above,None,20.0247224171,None


Note that a majority of the weights have been set to zero. So by setting an L1 penalty that's large enough, we are performing a subset selection. 

***QUIZ QUESTION***:
According to this list of weights, which of the features have been chosen? 

# Selecting an L1 penalty

To find a good L1 penalty, we will explore multiple values using a validation set. Let us do three way split into train, validation, and test sets:
* Split our sales data into 2 sets: training and test
* Further split our training data into two sets: train, validation

Be *very* careful that you use seed = 1 to ensure you get the same answer!

In [7]:
(training_and_validation, testing) = sales.random_split(.9,seed=1) # initial train/test split
(training, validation) = training_and_validation.random_split(0.5, seed=1) # split training into train and validate

Next, we write a loop that does the following:
* For `l1_penalty` in [10^1, 10^1.5, 10^2, 10^2.5, ..., 10^7] (to get this in Python, type `np.logspace(1, 7, num=13)`.)
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list.
    * Compute the RSS on VALIDATION data (here you will want to use `.predict()`) for that `l1_penalty`
* Report which `l1_penalty` produced the lowest RSS on validation data.

When you call `linear_regression.create()` make sure you set `validation_set = None`.

Note: you can turn off the print out of `linear_regression.create()` with `verbose = False`

In [8]:
create_lasso_model = lambda data, l1_penalty, features : graphlab.linear_regression.create(
    data, 
    features=features, 
    target='price', 
    l1_penalty=l1_penalty,
    l2_penalty=0.0, 
    validation_set=None,
    verbose=False
)

In [9]:
l1_penalty = np.logspace(1, 7, num=13)

In [10]:
models_and_pelanties=[]
for penalty in l1_penalty:
    models_and_pelanties.append({
        'penalty': penalty,
        'model':create_lasso_model(training, penalty, all_features)
    })

In [11]:
rss_data={}
for item in models_and_pelanties:
    model = item['model']
    penalty = item['penalty']
    predictions = model.predict(validation)
    residuals = predictions - validation['price']
    RSS = (residuals ** 2).sum()
    rss_data[str(penalty)] = RSS
    print 'RSS for penalty %.2f is: %.10E' %(penalty, RSS)

RSS for penalty 10.00 is: 6.2576628514E+14
RSS for penalty 31.62 is: 6.2576628536E+14
RSS for penalty 100.00 is: 6.2576628606E+14
RSS for penalty 316.23 is: 6.2576628826E+14
RSS for penalty 1000.00 is: 6.2576629521E+14
RSS for penalty 3162.28 is: 6.2576631721E+14
RSS for penalty 10000.00 is: 6.2576638676E+14
RSS for penalty 31622.78 is: 6.2576660675E+14
RSS for penalty 100000.00 is: 6.2576730279E+14
RSS for penalty 316227.77 is: 6.2576950764E+14
RSS for penalty 1000000.00 is: 6.2577651773E+14
RSS for penalty 3162277.66 is: 6.2579906285E+14
RSS for penalty 10000000.00 is: 6.2588371909E+14


In [73]:
get_min = lambda d: min(d, key=d.get)
print get_min(rss_data), ' : ', rss_data[get_min(rss_data)]

10000000.0  :  1.56983602382e+14


*** QUIZ QUESTION. *** What was the best value for the `l1_penalty`?

In [13]:
best_l1 = float(get_min(rss_data))
best_l1

10.0

***QUIZ QUESTION***
Also, using this value of L1 penalty, how many nonzero weights do you have?

In [14]:
model = create_lasso_model(training, best_l1, all_features)

In [15]:
model.coefficients.print_rows(num_rows=len(all_features))

+------------------+-------+------------------+--------+
|       name       | index |      value       | stderr |
+------------------+-------+------------------+--------+
|   (intercept)    |  None |  18993.4272128   |  None  |
|     bedrooms     |  None |  7936.96767903   |  None  |
| bedrooms_square  |  None |  936.993368193   |  None  |
|    bathrooms     |  None |  25409.5889341   |  None  |
|   sqft_living    |  None |  39.1151363797   |  None  |
| sqft_living_sqrt |  None |  1124.65021281   |  None  |
|     sqft_lot     |  None | 0.00348361822299 |  None  |
|  sqft_lot_sqrt   |  None |  148.258391011   |  None  |
|      floors      |  None |   21204.335467   |  None  |
|  floors_square   |  None |  12915.5243361   |  None  |
|    waterfront    |  None |  601905.594545   |  None  |
|       view       |  None |  93312.8573119   |  None  |
|    condition     |  None |  6609.03571245   |  None  |
|      grade       |  None |  6206.93999188   |  None  |
|    sqft_above    |  None |  4

***QUIZ QUESTION.*** What is the RSS on TEST data of the model with the best `l1_penalty`?

In [16]:
rss_data_test={}
predictions = model.predict(testing)
residuals = predictions - testing['price']
RSS = (residuals ** 2).sum()
rss_data[str(penalty)] = RSS
print 'RSS for penalty %.2f is: %.10E' %(penalty, RSS)

RSS for penalty 10000000.00 is: 1.5698360238E+14


# Limit the number of nonzero weights

What if we absolutely wanted to limit ourselves to, say, 7 features? This may be important if we want to derive "a rule of thumb" --- an interpretable model that has only a few features in them.

In this section, you are going to implement a simple, two phase procedure to achive this goal:
1. Explore a large range of `l1_penalty` values to find a narrow region of `l1_penalty` values where models are likely to have the desired number of non-zero weights.
2. Further explore the narrow region you found to find a good value for `l1_penalty` that achieves the desired sparsity.  Here, we will again use a validation set to choose the best value for `l1_penalty`.

In [17]:
max_nonzeros = 7

## Exploring the larger range of values to find a narrow range with the desired sparsity

Let's define a wide range of possible `l1_penalty_values`:

In [18]:
l1_penalty_values = np.logspace(8, 10, num=20)

Now, implement a loop that search through this space of possible `l1_penalty` values:

* For `l1_penalty` in `np.logspace(8, 10, num=20)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list. When you call `linear_regression.create()` make sure you set `validation_set = None`
    * Extract the weights of the model and count the number of nonzeros. Save the number of nonzeros to a list.
        * *Hint: `model['coefficients']['value']` gives you an SArray with the parameters you learned.  If you call the method `.nnz()` on it, you will find the number of non-zero parameters!* 

In [20]:
models_and_sparsity=[]
for penalty in np.logspace(8, 10, num=20):
    models_and_sparsity.append({
        'penalty': penalty,
        'sparsity':create_lasso_model(training, penalty, all_features)['coefficients']['value'].nnz()
    })

In [21]:
models_and_sparsity

[{'penalty': 100000000.0, 'sparsity': 18},
 {'penalty': 127427498.57031322, 'sparsity': 18},
 {'penalty': 162377673.91887242, 'sparsity': 18},
 {'penalty': 206913808.111479, 'sparsity': 18},
 {'penalty': 263665089.87303555, 'sparsity': 17},
 {'penalty': 335981828.6283788, 'sparsity': 17},
 {'penalty': 428133239.8719396, 'sparsity': 17},
 {'penalty': 545559478.1168514, 'sparsity': 17},
 {'penalty': 695192796.1775591, 'sparsity': 17},
 {'penalty': 885866790.4100832, 'sparsity': 16},
 {'penalty': 1128837891.6846883, 'sparsity': 15},
 {'penalty': 1438449888.2876658, 'sparsity': 15},
 {'penalty': 1832980710.8324375, 'sparsity': 13},
 {'penalty': 2335721469.0901213, 'sparsity': 12},
 {'penalty': 2976351441.6313133, 'sparsity': 10},
 {'penalty': 3792690190.7322536, 'sparsity': 6},
 {'penalty': 4832930238.571753, 'sparsity': 5},
 {'penalty': 6158482110.6602545, 'sparsity': 3},
 {'penalty': 7847599703.514623, 'sparsity': 1},
 {'penalty': 10000000000.0, 'sparsity': 1}]

Out of this large range, we want to find the two ends of our desired narrow range of `l1_penalty`.  At one end, we will have `l1_penalty` values that have too few non-zeros, and at the other end, we will have an `l1_penalty` that has too many non-zeros.  

More formally, find:
* The largest `l1_penalty` that has more non-zeros than `max_nonzeros` (if we pick a penalty smaller than this value, we will definitely have too many non-zero weights)
    * Store this value in the variable `l1_penalty_min` (we will use it later)
* The smallest `l1_penalty` that has fewer non-zeros than `max_nonzeros` (if we pick a penalty larger than this value, we will definitely have too few non-zero weights)
    * Store this value in the variable `l1_penalty_max` (we will use it later)


*Hint: there are many ways to do this, e.g.:*
* Programmatically within the loop above
* Creating a list with the number of non-zeros for each value of `l1_penalty` and inspecting it to find the appropriate boundaries.

In [69]:
get_gt = lambda d, thresh: [float(item['penalty']) for item in d if float(item['sparsity'] > thresh)]
get_lt = lambda d, thresh: [float(item['penalty']) for item in d if float(item['sparsity'] < thresh)]

In [68]:
l1_penalty_min = max(get_gt(models_and_sparsity, max_nonzeros))
l1_penalty_max = min(get_lt(models_and_sparsity, max_nonzeros))
print 'l1_penalty_min : %s' %l1_penalty_min
print 'l1_penalty_max : %s' %l1_penalty_max

l1_penalty_min : 2976351441.63
l1_penalty_max : 3792690190.73


***QUIZ QUESTION.*** What values did you find for `l1_penalty_min` and `l1_penalty_max`, respectively? 

## Exploring the narrow range of values to find the solution with the right number of non-zeros that has lowest RSS on the validation set 

We will now explore the narrow region of `l1_penalty` values we found:

In [70]:
l1_penalty_values = np.linspace(l1_penalty_min,l1_penalty_max,20)

* For `l1_penalty` in `np.linspace(l1_penalty_min,l1_penalty_max,20)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list. When you call `linear_regression.create()` make sure you set `validation_set = None`
    * Measure the RSS of the learned model on the VALIDATION set

Find the model that the lowest RSS on the VALIDATION set and has sparsity *equal* to `max_nonzeros`.

In [84]:
models_and_pelanties=[]
for penalty in l1_penalty_values:
    models_and_pelanties.append({
        'penalty': penalty,
        'model':create_lasso_model(training, penalty, all_features)
    })

In [87]:
rss_data_narrow={}
for item in models_and_pelanties:
    model = item['model']
    penalty = item['penalty']
    predictions = model.predict(validation)
    residuals = predictions - validation['price']
    RSS = (residuals ** 2).sum()
    sparsity = model['coefficients']['value'].nnz()
    rss_data_narrow[str(penalty)] = { 'RSS' : RSS, 'sparsity': sparsity }
    print 'RSS for penalty %.2f is: %.10E sparsity %d' %(penalty, RSS, sparsity)

RSS for penalty 2976351441.63 is: 9.6692569236E+14 sparsity 10
RSS for penalty 3019316638.95 is: 9.7401945008E+14 sparsity 10
RSS for penalty 3062281836.27 is: 9.8118836794E+14 sparsity 10
RSS for penalty 3105247033.59 is: 9.8932834246E+14 sparsity 10
RSS for penalty 3148212230.92 is: 9.9878321127E+14 sparsity 10
RSS for penalty 3191177428.24 is: 1.0084771670E+15 sparsity 10
RSS for penalty 3234142625.56 is: 1.0182987806E+15 sparsity 10
RSS for penalty 3277107822.88 is: 1.0282479922E+15 sparsity 10
RSS for penalty 3320073020.20 is: 1.0346169092E+15 sparsity 8
RSS for penalty 3363038217.52 is: 1.0385547359E+15 sparsity 8
RSS for penalty 3406003414.84 is: 1.0432372379E+15 sparsity 8
RSS for penalty 3448968612.16 is: 1.0469374888E+15 sparsity 7
RSS for penalty 3491933809.48 is: 1.0511476256E+15 sparsity 7
RSS for penalty 3534899006.81 is: 1.0559927353E+15 sparsity 7
RSS for penalty 3577864204.13 is: 1.0607995318E+15 sparsity 7
RSS for penalty 3620829401.45 is: 1.0657076895E+15 sparsity 6


***QUIZ QUESTIONS***
1. What value of `l1_penalty` in our narrow range has the lowest RSS on the VALIDATION set and has sparsity *equal* to `max_nonzeros`?

In [150]:
rss_with_sparsity_of = lambda d, sparsity: dict((k, v) for k, v in d.items() if v['sparsity'] == sparsity)

In [151]:
rss_with_sparsity_of(rss_data_narrow, 7)

{'3448968612.163437': {'RSS': 1046937488751711.5, 'sparsity': 7},
 '3491933809.484539': {'RSS': 1051147625612862.5, 'sparsity': 7},
 '3534899006.805641': {'RSS': 1055992735342999.4, 'sparsity': 7},
 '3577864204.126743': {'RSS': 1060799531763286.9, 'sparsity': 7}}

In [153]:
best_l1_value = 3491933809.484539
best_l1_value

3491933809.484539

***QUIZ QUESTIONS***
2. What features in this model have non-zero coefficients?

In [154]:
model = create_lasso_model(training, best_l1_value, all_features)
model.coefficients[model.coefficients['value']!= 0]

name,index,value,stderr
(intercept),None,224545.136501,None
bedrooms,None,496.983429977,None
bathrooms,None,15640.8229131,None
sqft_living,None,32.2039341994,None
sqft_living_sqrt,None,678.904419357,None
grade,None,2825.4694254,None
sqft_above,None,29.715599776,None
